In [5]:
from flask import Flask

app = Flask(__name__)

@app.before_first_request
def before_first():
    print("This runs before the first request.")

@app.route('/')
def home():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(debug=True)


ImportError: cannot import name 'escape' from 'jinja2' (C:\Users\D ARUNA PRIYA\AppData\Roaming\Python\Python39\site-packages\jinja2\__init__.py)

In [4]:
#converting json to Squad data
import json

# Load your JSON data
with open(r'C:\Users\D ARUNA PRIYA\OneDrive\Desktop\BTech\Projects\Applicative Project - 1\summaries.json', 'r') as f:
    data = json.load(f)

# Convert to SQuAD format
squad_data = {'data': []}
for item in data:
    squad_data['data'].append({
        'title': 'Custom Dataset',
        'paragraphs': [{
            'context': item['context'],
            'qas': [{
                'id': str(hash(item['question'])),  # Unique ID for each question
                'question': item['question'],
                'answers': [{'text': item['answer'], 'answer_start': item['context'].find(item['answer'])}],
                'is_impossible': False
            }]
        }]
    })

# Save as a new JSON file
with open('custom_squad_format.json', 'w') as f:
    json.dump(squad_data, f)


In [3]:
#fine tuning the distilBERT model on custom dataset

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

# Load tokenizer and model
model_name = "distilbert-base-uncased"  # Or any other question-answering model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Load the custom dataset in SQuAD format
train_dataset = load_dataset('json', data_files='custom_squad_format.json', field='data')

# Tokenize the dataset
def preprocess(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        answer = answers[i]["text"]
        start_char = answers[i]["answer_start"]
        end_char = start_char + len(answer)

        sequence_ids = inputs.sequence_ids(i)
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

        if offsets[context_start][0] <= start_char and offsets[context_end][1] >= end_char:
            start_token = next(
                idx for idx, (start, end) in enumerate(offsets)
                if start <= start_char < end
            )
            end_token = next(
                idx for idx, (start, end) in enumerate(offsets)
                if start < end_char <= end
            )

            start_positions.append(start_token)
            end_positions.append(end_token)
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Preprocess the dataset
tokenized_dataset = train_dataset.map(preprocess, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_qa_model",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_qa_model")
tokenizer.save_pretrained("./fine_tuned_qa_model")


ModuleNotFoundError: No module named 'transformers'

In [ ]:
#using fine tuned model in the flask app

from transformers import pipeline

# Load the fine-tuned model and tokenizer
qa_pipeline = pipeline("question-answering", model="./fine_tuned_qa_model", tokenizer="./fine_tuned_qa_model")
